In [ ]:
!pip install langchain openai tiktoken faiss-cpu

##Import necessary libraries

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import getpass

##OpenAI API Key

In [ ]:
os.environ["OPENAI_API_KEY"] = "*********"#Enter your API Key

##Providing Docs for retrieval

In [ ]:
academic_text = """
LangChain is a framework designed to help developers build powerful applications that use large language models (LLMs).
It provides tools for chaining different components like prompts, models, and memory for conversational systems.
Retrieval-Augmented Generation (RAG) helps improve accuracy by combining external knowledge with LLM responses.
Prompting techniques like Few-Shot Learning and Chain-of-Thought (CoT) reasoning can improve reasoning and factual accuracy.
"""
with open("academic_faq.txt", "w") as f:
    f.write(academic_text)

##Creating Embeddings for Documents

In [ ]:
loader = TextLoader("academic_faq.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(split_docs, embeddings)

##Initializaing Memory

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model="gpt-4o-mini", temperature=0.3),
    retriever=vectorstore.as_retriever(),
    memory=memory
)

##Zero Shot prompting

In [ ]:
zero_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer this question as clearly as possible:\nQuestion: {question}"
)


##Few shot prompting

In [ ]:
few_shot_examples = """
Example 1:
Q: What is LangChain?
A: A framework for building applications using LLMs.

Example 2:
Q: What is RAG?
A: Retrieval-Augmented Generation improves factual accuracy.

Now, answer:
Q: {question}
"""
few_shot_prompt = PromptTemplate(input_variables=["question"], template=few_shot_examples)

##Chain of thought prompting

In [ ]:
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="Think step by step before answering. Question: {question}")

In [ ]:
self_reflection_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a reasoning agent. First, give an initial answer. Then, reflect and correct it if needed.\n"
        "Question: {question}\n"
        "Answer:"
    )
)

##Initializing llm and langchain chains

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

zero_chain = LLMChain(llm=llm, prompt=zero_prompt)
few_chain = LLMChain(llm=llm, prompt=few_shot_prompt)
cot_chain = LLMChain(llm=llm, prompt=cot_prompt)
self_reflect_chain = LLMChain(llm=llm, prompt=self_reflection_prompt)

In [ ]:
question = "How does RAG improve chatbot accuracy?"

print("Zero-shot:")
print(zero_chain.run(question))
print("\nFew-shot:")
print(few_chain.run(question))
print("\nChain-of-Thought:")
print(cot_chain.run(question))
print("\nSelf-Reflection:")
print(self_reflect_chain.run(question))